In [2]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, applications
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import time
from tensorflow.keras import backend as k
import csv

In [2]:
tf.keras.backend.set_image_data_format('channels_last')

In [3]:
categories = ['Toddler', 'Teenagers', 'Adults']
CLASSES = len(categories)

In [4]:
PATH = 'data'
train_path = os.path.join(PATH, 'train')
IMG_SIZE=224

In [5]:
ep = 12        # Epochs
bs = 32         # Batch size

In [6]:
X = np.load('features.npy')
y = np.load('labels.npy')

In [7]:
X.shape

(2803, 224, 224, 3)

In [8]:
X = X/255.0      # Normalize
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

## VGG16 all frozen layers and Additional Conv layers

In [4]:
NAME = "friendship-day-classifier-{}".format(time.strftime("%d%h-%m-%S"))
tensorboard = TensorBoard(log_dir="""logs\{}""".format(NAME))

def cnn_model(X_train, X_test , y_train, y_test):
    
    base_model = applications.VGG16(include_top=False, input_shape=X_train.shape[1:], weights='imagenet',classes=CLASSES)

    # Freezing VGG16 layers
    for layer in base_model.layers:
        layer.trainable=False
    
    last_layer = 'block5_pool'
    model = Model(base_model.input, base_model.get_layer(last_layer).output)

    model.layers[-1].output.shape
    model = Sequential()

    model.add(base_model)      # Stack vgg16 

    model.add(Conv2D(128,(3,3),activation="relu", input_shape=model.layers[-1].output.shape, padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
    
    model.add(Conv2D(64,(2,2),activation="relu"))
    model.add(MaxPooling2D(2,2))
    
    model.add(Flatten())        # Flatten the output

    model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(CLASSES, activation="softmax"))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=bs, epochs=ep, validation_data = (X_test, y_test), callbacks=[tensorboard])

    # Save model
    model.save(NAME)

## VGG16, Last two layers unfrozen with 10 ep

In [3]:
NAME = "vgg-unfreeze-last-2"
tensorboard = TensorBoard(log_dir="""logs\{}""".format(NAME))

def cnn_model_3(X_train, X_test , y_train, y_test):
    
    base_model = applications.VGG16(include_top=False, input_shape=X_train.shape[1:], weights='imagenet',classes=CLASSES)

    # Freezing VGG16 layers
    for layer in base_model.layers:
        layer.trainable=False
    
    # Unfreezing last two layers
    base_model.layers[-1].trainable = True
    base_model.layers[-2].trainable = True
    
    last_layer = 'block5_pool'
    model = Model(base_model.input, base_model.get_layer(last_layer).output)

    model.layers[-1].output.shape
    model = Sequential()

    model.add(base_model)      # Stack vgg16 

#     model.add(Conv2D(128,(3,3),activation="relu", input_shape=model.layers[-1].output.shape, padding='same'))
#     model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
    
#     model.add(Conv2D(64,(2,2),activation="relu"))
#     model.add(MaxPooling2D(2,2))
    
    model.add(Flatten())        # Flatten the output

    model.add(Dense(12, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(CLASSES, activation="softmax"))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=16, epochs=ep, validation_data = (X_test, y_test), callbacks=[tensorboard])

    # Save model
    model.save(NAME)

In [9]:
NAME = "vgg-unfreeze-last-2-lr-ep20"
tensorboard = TensorBoard(log_dir="""logs\{}""".format(NAME))

def cnn_model_3(X_train, X_test , y_train, y_test):
    
    base_model = applications.VGG16(include_top=False, input_shape=X_train.shape[1:], weights='imagenet',classes=CLASSES)

    # Freezing VGG16 layers
    for layer in base_model.layers:
        layer.trainable=False
    
    # Unfreezing last two layers
    base_model.layers[-1].trainable = True
    base_model.layers[-2].trainable = True
    
    last_layer = 'block5_pool'
    model = Model(base_model.input, base_model.get_layer(last_layer).output)

    model.layers[-1].output.shape
    model = Sequential()

    model.add(base_model)      # Stack vgg16 

#     model.add(Conv2D(128,(3,3),activation="relu", input_shape=model.layers[-1].output.shape, padding='same'))
#     model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
    
#     model.add(Conv2D(64,(2,2),activation="relu"))
#     model.add(MaxPooling2D(2,2))
    
    model.add(Flatten())        # Flatten the output

    model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(CLASSES, activation="softmax"))

    model.compile(optimizer=Adam(learning_rate=0.0001) , loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=bs, epochs=ep, validation_data = (X_test, y_test), callbacks=[tensorboard])

    # Save model
    model.save(NAME)

In [10]:
#cnn_model_3(X_train, X_test, y_train, y_test)

Train on 2242 samples, validate on 561 samples
Epoch 1/20
2242/2242 [==============================] - 94s 42ms/sample - loss: 3.0244 - accuracy: 0.4670 - val_loss: 2.4724 - val_accuracy: 0.5775
Epoch 2/20
2242/2242 [==============================] - 76s 34ms/sample - loss: 2.1037 - accuracy: 0.6494 - val_loss: 1.9735 - val_accuracy: 0.6078
Epoch 3/20
2242/2242 [==============================] - 81s 36ms/sample - loss: 1.6912 - accuracy: 0.7132 - val_loss: 1.7245 - val_accuracy: 0.6364
Epoch 4/20
2242/2242 [==============================] - 84s 37ms/sample - loss: 1.3919 - accuracy: 0.8033 - val_loss: 1.5610 - val_accuracy: 0.6631
Epoch 5/20
2242/2242 [==============================] - 84s 37ms/sample - loss: 1.1511 - accuracy: 0.8680 - val_loss: 1.5499 - val_accuracy: 0.6524
Epoch 6/20
2242/2242 [==============================] - 84s 37ms/sample - loss: 0.9687 - accuracy: 0.9193 - val_loss: 1.4315 - val_accuracy: 0.6791
Epoch 7/20
2242/2242 [==============================] - 83s 37ms/

## VGG16 with last two unfrozen layers and Dense layers

In [9]:
NAME="frozen_vgg"
checkpoint_path = "saved_weights/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
def frozen_vgg(X_train, X_test , y_train, y_test):
    
    base_model = applications.VGG16(include_top=False, input_shape=X_train.shape[1:], weights='imagenet',classes=CLASSES)

    # Freezing VGG16 layers
    for layer in base_model.layers:
        layer.trainable=False
    
    base_model.layers[-1].trainable = True
    base_model.layers[-2].trainable = True
    
    last_layer = 'block5_pool'
    model = Model(base_model.input, base_model.get_layer(last_layer).output)

    model.layers[-1].output.shape
    model = Sequential()

    model.add(base_model)      # Stack vgg16 


    
    model.add(Flatten())        # Flatten the output

    model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(CLASSES, activation="softmax"))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=bs, epochs=ep, validation_data = (X_test, y_test), callbacks=[cp_callback])

    # Save model
    model.save(NAME)

In [10]:
frozen_vgg(X_train, X_test, y_train, y_test)

Train on 2242 samples, validate on 561 samples
Epoch 1/12
2240/2242 [============================>.] - ETA: 0s - loss: 2.1058 - accuracy: 0.4897
Epoch 00001: saving model to saved_weights/cp.ckpt
2242/2242 [==============================] - 98s 44ms/sample - loss: 2.1048 - accuracy: 0.4897 - val_loss: 1.3843 - val_accuracy: 0.5597
Epoch 2/12
2240/2242 [============================>.] - ETA: 0s - loss: 1.0993 - accuracy: 0.6415
Epoch 00002: saving model to saved_weights/cp.ckpt
2242/2242 [==============================] - 80s 36ms/sample - loss: 1.0993 - accuracy: 0.6414 - val_loss: 1.1324 - val_accuracy: 0.5704
Epoch 3/12
2240/2242 [============================>.] - ETA: 0s - loss: 0.8739 - accuracy: 0.7116
Epoch 00003: saving model to saved_weights/cp.ckpt
2242/2242 [==============================] - 80s 36ms/sample - loss: 0.8736 - accuracy: 0.7119 - val_loss: 1.0391 - val_accuracy: 0.6346
Epoch 4/12
2240/2242 [============================>.] - ETA: 0s - loss: 0.7172 - accuracy: 0.79

In [11]:
NAME="ftp-ep10"
def ftp(X_train, X_test , y_train, y_test):
    base_model = applications.VGG16(include_top=False, input_shape=X_train.shape[1:], weights=None,classes=CLASSES)

    # Freezing VGG16 layers
    for layer in base_model.layers:
        layer.trainable=False
    
    base_model.layers[-1].trainable = True
    base_model.layers[-2].trainable = True
    
    last_layer = 'block5_pool'
    model = Model(base_model.input, base_model.get_layer(last_layer).output)

    model.layers[-1].output.shape
    model = Sequential()

    model.add(base_model)      # Stack vgg16 

    
    model.add(Flatten())        # Flatten the output

    model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(CLASSES, activation="softmax"))
    model.compile(optimizer=Adam(learning_rate=0.0001) , loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.load_weights(checkpoint_path)
    model.fit(X_train, y_train, batch_size=bs, epochs=ep, validation_data = (X_test, y_test))

    # Save model
    model.save(NAME)

In [12]:
model = ftp(X_train, X_test, y_train, y_test)

Train on 2242 samples, validate on 561 samples
Epoch 1/12
2242/2242 [==============================] - 82s 36ms/sample - loss: 0.1539 - accuracy: 0.9920 - val_loss: 1.7895 - val_accuracy: 0.6185
Epoch 2/12
2242/2242 [==============================] - 78s 35ms/sample - loss: 0.1448 - accuracy: 0.9924 - val_loss: 1.5548 - val_accuracy: 0.6435
Epoch 3/12
2242/2242 [==============================] - 78s 35ms/sample - loss: 0.1295 - accuracy: 0.9920 - val_loss: 1.5364 - val_accuracy: 0.6399
Epoch 4/12
2242/2242 [==============================] - 81s 36ms/sample - loss: 0.1459 - accuracy: 0.9884 - val_loss: 1.6025 - val_accuracy: 0.6061
Epoch 5/12
2242/2242 [==============================] - 78s 35ms/sample - loss: 0.1623 - accuracy: 0.9888 - val_loss: 2.0215 - val_accuracy: 0.6542
Epoch 6/12
2242/2242 [==============================] - 78s 35ms/sample - loss: 0.1672 - accuracy: 0.9839 - val_loss: 1.7707 - val_accuracy: 0.6168
Epoch 7/12
2242/2242 [==============================] - 79s 35ms/

# Experimental tests


In [5]:
test_csv = "./data/test.csv"        # Path to test.csv file
test_images = "./data/test/"        # Path to test images folder

In [6]:
model = tf.keras.models.load_model('./ftp-model/')

In [7]:
def create_test_data():
    images = []       # List containes path of all the testing images in right seq
    test_data = []    
    df = pd.read_csv(test_csv)
    names_df = df['Filename']
    for img in names_df:
        img_path = os.path.join(test_images, img)
        images.append(img_path)        
    for img_path in images:
        try:
            img_array = cv2.imread(img_path ,cv2.COLOR_BGR2RGB)  # convert to array
            img_array = img_array/255.0
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
            test_data.append(new_array.reshape(IMG_SIZE, IMG_SIZE, 3))  # add this to our training_data

        except Exception as e:
            print("Error !!")
            print(e)
            pass
    return test_data

    
test_data = np.array(create_test_data())
print(test_data.shape)

(314, 224, 224, 3)


In [8]:
prediction = model.predict(test_data)

In [9]:
results = []
for i in prediction:
    results.append(np.argmax(i))
    
results[:5]

[0, 2, 1, 1, 1]

In [10]:
def save_results(results):
    images = pd.read_csv(test_csv)['Filename']
    with open('ftp-model.csv', 'w', newline='') as f:
        w = csv.writer(f)
        w.writerow(["Filename", "Category"])
        for i in range(len(images)):
            res = str(categories[results[i]])
            img = images[i]
            w.writerow([img, res])
            
save_results(results)